In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies= pd.read_csv('movies_metadata.csv', low_memory=False)
links= pd.read_csv('links.csv')
credits=pd.read_csv('credits.csv')
keywords=pd.read_csv('keywords.csv')
ratings=pd.read_csv('ratings.csv')

In [ ]:
movies.head()

In [ ]:

invalid_rows = movies[~movies['id'].apply(pd.to_numeric, errors='coerce').notnull()]
print(invalid_rows)


In [ ]:
non_numeric_ids = movies[~movies['id'].str.isnumeric()]['id']

print(non_numeric_ids)

movies = movies[movies['id'].str.isnumeric()]

movies['id'] = movies['id'].astype('int64')

merged_df = pd.merge(movies, credits, on='id', how='left')

print(merged_df.head())


In [ ]:
movies = movies.copy()

movies['id'] = movies['id'].astype(str)

movies = movies[movies['id'].str.isnumeric()]

movies['id'] = movies['id'].astype('int64')

merged_df = pd.merge(movies, credits, on='id', how='left')

print(merged_df.head())


In [ ]:
merged_df = pd.merge(merged_df, keywords, on='id', how='left')



In [ ]:

print(links.columns)


In [ ]:
merged_df = pd.merge(merged_df, links, left_on='id', right_on='movie_id', how='left')



In [ ]:
merged_df = pd.merge(merged_df, ratings, on='movie_id', how='left')

In [ ]:
merged_df.head()

In [ ]:
merged_df = merged_df[['id', 'original_title', 'overview','genres', 'cast', 'crew']]
merged_df.head()


In [ ]:
merged_df.isnull()

In [ ]:
merged_df = merged_df[['id', 'original_title', 'overview','genres', 'cast', 'crew']].copy()
merged_df.dropna(inplace=True)


In [ ]:
merged_df.duplicated().sum()

In [ ]:
merged_df = merged_df.drop_duplicates()


In [ ]:
merged_df.iloc[0].genres

In [ ]:
import ast

def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
merged_df['genres']=merged_df['genres'].apply(convert)
merged_df['genres']

In [ ]:
merged_df.head()

In [ ]:
def convert2(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
         L.append(i['name'])
         counter+1
        else:
           break
    return L

In [ ]:
merged_df['cast']=merged_df['cast'].apply(convert2)
merged_df.head()

In [ ]:
import ast

merged_df["crew"] = merged_df["crew"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [ ]:
def get_director(crew_list):
    if isinstance(crew_list, list):  # Check if it's a list
        for member in crew_list:
            if member.get("job") == "Director": 
                return member.get("name") 
    return None  

In [ ]:
merged_df["crew"] = merged_df["crew"].apply(get_director)
merged_df['crew']

In [ ]:
merged_df.head()

In [ ]:

for col in ["overview","genres", "cast", "crew"]:
    merged_df[col] = merged_df[col].astype(str)# Convert list-type columns to strings

merged_df = merged_df.drop_duplicates()


In [ ]:
merged_df.head()

In [ ]:
merged_df['overview'] = merged_df['overview'].apply(lambda x: x.split() if isinstance(x, str) else x)


merged_df['overview']

In [ ]:
merged_df.head()

In [ ]:
merged_df['tags'] = merged_df['genres'].astype(str) + " " + \
                    merged_df['cast'].astype(str) + " " + \
                    merged_df['crew'].astype(str)


In [ ]:
merged_df.head()

In [ ]:
new_df=merged_df[['id','original_title','tags']]

In [ ]:
new_df.head()

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem (text):
    y=[]
    for i in text.split():
     y.append(ps.stem (i))
    return " ".join(y)

In [ ]:
new_df.loc[:,'tags']= new_df['tags'].apply(stem)

In [ ]:
new_df['tags']

In [ ]:
import re

def clean_tags(tag):
    if isinstance(tag, str):
        # Extract only the words inside square brackets
        matches = re.findall(r"\[([^\]]+)\]", tag)
        if matches:
            # Flatten list and join as a single string
            return " ".join([item.replace("'", "").strip() for sublist in matches for item in sublist.split(",")])
    return str(tag)

new_df.loc[:, 'tags'] = new_df['tags'].apply(clean_tags)


In [ ]:
new_df['tags'][0]

In [ ]:
new_df.loc[:, 'tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df['tags']

In [ ]:
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

vector = cv.fit_transform(new_df['tags'].to_numpy())  


In [ ]:
vector = cv.fit_transform(new_df['tags'].to_numpy())  



In [ ]:
print(vector.toarray()) 


In [ ]:
vector.shape


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


cv = CountVectorizer(stop_words='english')
vector = cv.fit_transform(new_df['tags'])

# Compute similarity matrix
similarity = cosine_similarity(vector)

print(similarity.shape)  # Verify similarity matrix size


In [ ]:
def recommend(movies):
    movie_index=new_df[new_df['original_title']==movies].index[0]
    distances= similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(new_df.iloc[i[0]].original_title)



In [ ]:
recommend('Toy Story')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pk1','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pk1','wb'))

In [ ]:
print(type(similarity))  # <class 'numpy.ndarray'>
print(similarity.shape)  # (num_movies, num_movies)


In [ ]:
import psutil
print(f"Memory Usage: {psutil.virtual_memory().percent}%")


In [ ]:
import pickle


movies = pickle.load(open('movies.pk1', 'rb'))

print("Columns in movies dataset:", movies.columns)


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

movies = pickle.load(open('movies.pk1', 'rb'))


if 'tags' not in movies.columns:
    raise KeyError(" Error: 'tags' column not found in dataset!")

vectorizer = TfidfVectorizer(stop_words='english')
vector = vectorizer.fit_transform(movies['tags'].fillna(''))

pickle.dump(vector, open('vector.pk1', 'wb'))
print(" vector.pk1 generated successfully!")



In [ ]:

from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix


movies = pickle.load(open('movies.pk1', 'rb'))
vector = pickle.load(open('vector.pk1', 'rb'))

# If vector is sparse, keep it sparse
if isinstance(vector, csr_matrix):
    vector = vector.tocsr()

#Use Nearest Neighbors for fast search
nn_model = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')
nn_model.fit(vector)


precomputed_recommendations = {}

for i in range(len(movies)):
    distances, indices = nn_model.kneighbors(vector[i], return_distance=True)
    precomputed_recommendations[movies.iloc[i].original_title] = [
        movies.iloc[j].original_title for j in indices[0][1:]  # Skip first (itself)
    ]


with open('precomputed_recommendations.pk1', 'wb') as f:
    pickle.dump(precomputed_recommendations, f)

print(" Precomputed recommendations saved successfully!")


In [ ]:
from fuzzywuzzy import process

# Match movie title to the closest name in dataset
def get_best_match(movie_name):
    matches = process.extractOne(movie_name, movies['original_title'].values)
    return matches[0] if matches else None

#Update Recommendation Function
def recommend(movie):
    best_match = get_best_match(movie)  # Find the closest matching movie
    if not best_match:
        return ["No matching movie found."]

    movie_index = movies[movies['original_title'] == best_match].index[0]
    distances, indices = nn_model.kneighbors(vector[movie_index], return_distance=True)
    
    return [movies.iloc[j].original_title for j in indices[0][1:]]  # Skip first (itself)
